In [ ]:
from dolfin import *
from mshr import *
from rbnics.backends.dolfin.wrapping import counterclockwise
from rbnics.shape_parametrization.utils.symbolic import VerticesMappingIO

In [ ]:
# Define domain
primary = Rectangle(Point(-0.15, 0), Point(0.15, 0.6))
left = Rectangle(Point(-0.65, 0.4), Point(-0.15, 0.6))
right = Rectangle(Point(0.15, 0.4), Point(0.65, 0.6))
domain = primary + left + right

In [ ]:
# Vertices mapping (8 subdomains)
vertices_mappings = [
    {
        ("-0.15", "0.0"): ("-mu[0]/2", "0.0"),
        ("0.15", "0.0"): ("mu[0]/2", "0.0"),
        ("-0.15", "0.4"): ("-mu[0]/2", "mu[1]")
    }, # subdomain 1
    {
        ("0.15", "0.0"): ("mu[0]/2", "0.0"),
        ("-0.15", "0.4"): ("-mu[0]/2", "mu[1]"),
        ("0.15", "0.4"): ("mu[0]/2","mu[1]")
    }, # subdomain 2
    {
        ("-0.15", "0.4"): ("-mu[0]/2", "mu[1]"),
        ("0.15", "0.4"): ("mu[0]/2", "mu[1]"),
        ("-0.15", "0.6"): ("-mu[0]/2", "mu[1]+mu[2]")
    }, # subdomain 3
    {
        ("0.15", "0.4"): ("mu[0]/2", "mu[1]"),
        ("-0.15", "0.6"): ("-mu[0]/2", "mu[1]+mu[2]"),
        ("0.15", "0.6"): ("mu[0]/2", "mu[1]+mu[2]")
    },  # subdomain 4
    {
        ("0.15", "0.6"): ("mu[0]/2", "mu[1]+mu[2]"),
        ("0.15", "0.4"): ("mu[0]/2", "mu[1]"),
        ("0.65", "0.4"): ("mu[0]/2+mu[3]", "mu[1]")
    },  # subdomain 5
    {
        ("0.15", "0.6"): ("mu[0]/2", "mu[1]+mu[2]"),
        ("0.65", "0.4"): ("mu[0]/2+mu[3]", "mu[1]"),
        ("0.65", "0.6"): ("mu[0]/2+mu[3]", "mu[1]+mu[2]")
    },  # subdomain 6
    {
        ("-0.65", "0.6"): ("-mu[0]/2-mu[4]", "mu[1]+mu[2]"),
        ("-0.65", "0.4"): ("-mu[0]/2-mu[4]", "mu[1]"),
        ("-0.15", "0.4"): ("-mu[0]/2", "mu[1]")
    },  # subdomain 7
    {
        ("-0.65", "0.6"): ("-mu[0]/2-mu[4]", "mu[1]+mu[2]"),
        ("-0.15", "0.6"): ("-mu[0]/2", "mu[1]+mu[2]"),
        ("-0.15", "0.4"): ("-mu[0]/2", "mu[1]")
    }  # subdomain 8
]

In [ ]:
# Create mesh
for i, vertices_mapping in enumerate(vertices_mappings):
    subdomain_i = Polygon([Point(*[float(coord) for coord in vertex]) for vertex in counterclockwise(vertices_mapping.keys())])
    domain.set_subdomain(i+1, subdomain_i)
mesh = generate_mesh(domain,50)
plot(mesh)

In [ ]:
# Create subdomains
subdomains = MeshFunction("size_t", mesh, 2, mesh.domains())
plot(subdomains)

In [ ]:
# Create boundaries

tol = DOLFIN_EPS

# primary fin
class Inlet(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[1]) < tol
class PBottomRight(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[0]-0.15) < tol and x[1] <= 0.4
class PTopRight(SubDomain):
    def inside(self, x, on_boundary):
        return abs(x[0]-0.15) < tol and x[1] >= 0.4
class PBottomLeft(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[0]+0.15) < tol and x[1] <= 0.4
class PTopLeft(SubDomain):
    def inside(self, x, on_boundary):
        return abs(x[0]+0.15) < tol and x[1] >= 0.4
class PTop(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[1]-0.6) < tol and x[0] >= -0.15 and x[0] <= 0.15
class PBottom(SubDomain):
    def inside(self, x, on_boundary):
        return abs(x[1]-0.4) < tol and x[0] >= -0.15 and x[0] <= 0.15

# right subfin
class RBottom(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and x[0] >= 0.15 and abs(x[1]-0.4) < tol
class RRight(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[0]-0.65) < tol 
class RTop(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and x[0] >= 0.15 and abs(x[1]-0.6) < tol 
class RLeft(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[0]-0.15) < tol and x[1] >= 0.4

# left subfin
class LBottom(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and x[0] <= -0.15 and abs(x[1]-0.4) < tol
class LRight(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[0]+0.15) < tol and x[1] >= 0.4
class LTop(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and x[0] <= -0.15 and abs(x[1]-0.6) < tol
class LLeft(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary and abs(x[0]+0.65) < tol

boundaries = MeshFunction("size_t", mesh, mesh.topology().dim()-1)
boundaries.set_all(0)
inlet = Inlet()
inlet.mark(boundaries,1)
pBottomRight = PBottomRight()
pBottomRight.mark(boundaries,2)
rBottom = RBottom()
rBottom.mark(boundaries,3)
rRight = RRight()
rRight.mark(boundaries,4)
rTop = RTop()
rTop.mark(boundaries,5)
pTop = PTop()
pTop.mark(boundaries,6)
lTop = LTop()
lTop.mark(boundaries,7)
lLeft = LLeft()
lLeft.mark(boundaries, 8)
lBottom = LBottom()
lBottom.mark(boundaries,9)
pBottomLeft = PBottomLeft()
pBottomLeft.mark(boundaries,10)
pTopRight = PTopRight()
pTopRight.mark(boundaries,11)
pTopLeft = PTopLeft()
pTopLeft.mark(boundaries,12)
pBottom = PBottom()
pBottom.mark(boundaries,13)

In [ ]:
# Save
VerticesMappingIO.save_file(vertices_mappings, ".", "subfin1_vertices_mapping.vmp")
File("subfin1.xml") << mesh
File("subfin1_physical_region.xml") << subdomains
File("subfin1_facet_region.xml") << boundaries
XDMFFile("subfin1.xdmf").write(mesh)
XDMFFile("subfin1_physical_region.xdmf").write(subdomains)
XDMFFile("subfin1_facet_region.xdmf").write(boundaries)